In [1]:
### Vortex Automation TODO list

### Steps breakdown

### Gather data

    ### TODO Download Vortex through webdriver (selenium)
    ### TODO move data into project folder
    ### Import data into pandas for processing

### Process data
    
    ### utilize previous data process in the vortex process
    ### do Meghan's summing year over year process for delivery
    ### assemble into final sending excel sheet
    ### TODO Create graphs that are alread included into the current process
    ### TODO output graphs
    
### Send deliverable

    ### TODO Send email with deliverable to appropriate points of contact
    
### Additional Work (After completion of V1)

    ### Create automated EXE for process in Pycharm?
    ### Fully automate interface for dowloading/assembling and sending
    ### add monitoring for the output files to decide to send or not.

"""
Start Here!

1. Paste raw .nc files into the inputs section of this tool
2. press double play button above, restart the kernel, then re-run
3. Check output file with tables included at the bottom of this tool
4. Output file with be placed in the outputs folder of the tool
"""

In [2]:
import os
import netCDF4 as nc
import pandas as pd
from datetime import datetime, timedelta, date
from openpyxl import Workbook, utils
from openpyxl.styles import Alignment, NamedStyle
from openpyxl.utils.dataframe import dataframe_to_rows
import conversion
from dateutil.relativedelta import relativedelta

In [3]:
def main():

    """
    Main function to drive process
    1. web driver (unless we get API access)
    2. Data import
    3. Data processing
    4. Produce images/graphs
    5. Assemble in excel deliverable
    6. Send
    
    Parameters
    ----------
        vortex web inputs (if needed)
            some inputs might be needed like a date or location for the vortex pulls. 
            Will probably be a single list of location we use that will be updated when adjusted or a 
            new project is used for monitoring
        
    
    """
    
    # define dict with project names and keys
    # TEMP energy sensitivities for each of the project pulled manually, until implemented hindcast code
    projects = {
        "410075" : {'name':'Willow Creek', 'sens': 1.11},
        '410077' : {'name':"Plum Creek", 'sens': 1.23},
        '431561' : {"name":"Dermott", 'sens': 1.48},
        '431563' : {"name":"Willow Springs", 'sens': 1.55},
        '431565' : {"name":"Lockett", 'sens': 1.37},
        '431567' : {"name":"Tahoka", 'sens': 1.33},
        '410079' : {"name":"Sage Draw", 'sens': 1.39},
        '433343' : {"name":"Helena", 'sens': 1.95},
        '433345' : {"name":"Haystack", 'sens': 1.4},
        '433347' : {"name":"Lincoln Land", 'sens': 1.44},
        '433341' : {"name":"Western Trail", 'sens': 1.6},
        '767' : {"name":"Ireland Southwest"},
        '769' : {"name":"Ireland Southeast"},
        '770' : {"name":"Ireland North"},
        '777' : {"name":"Kennoxhead"},
        '542391' : {"name":"Ford Ridge", 'sens': 1.294188},
        '549447' : {"name":"Sunflower", 'sens': 1.49},
    }
    
    # import datafiles
    
    # Get inputs folder path
    wd = os.getcwd()
    inputs_folder_path = os.path.join(wd, 'inputs')
    
    # Call import function
    data_list, data_dict = netCDF_import(inputs_folder_path, projects)
    
    # process data
    excel_data = data_processing(data_list)
    
    # windiness and engerginess, pass each dataframe individually for process
    for project in excel_data:
        project['data'] = one_month_windiness(project['data'])
        project['data'] = three_month_windiness(project['data'])
        project['data'] = twelve_month_windiness(project['data'])
        project = energiness(project)
        project = iav(project)
    
    # Excel output function for Denmark deliverable
    excel_output(excel_data)
    
    # Excel output for internal analysis
    excel_internal_output(excel_data)

    return excel_data

In [4]:
def netCDF_import(inputs_dir, projects):
    """
    import the datafiles from the inputs folder into python for processing
    """
    projects_list = []
    files = []
    data = {}
    ### Import data into pandas for processing
    for file in os.listdir(inputs_dir):
        # check for files in the correct format only
        if file.endswith(".nc"):
            files.append(f"./inputs/{file}")
    
    # if input files are not in folder, exit code
    if len(files) == 0:
        print("Inputs folder does not include .nc files, please add inputs and try again.")
        return
    
    else:
        # iterate through filelist discovered above
        for file in files:
            
            ds = nc.Dataset(file)  #import dataset with nc library 
            M = ds['M'][:, :, 0, 0] 
            lev = ds['lev'][:]
            time = ds['time'][:]

            df_M = pd.DataFrame(data=M, columns=['50', '60', '70', '80', '90', '100', '110', '120', '130', '140', '150', '160', '170', '180', '190', '200',
                                                     '220', '240', '260', '280', '300'])  # rename cols to hub heights
            df_lev = pd.DataFrame(data=lev)
            df_time = pd.DataFrame(data=time, columns=['PosixTime'])
            df_time['PosixTime'] = df_time['PosixTime'] * 3600 # Convert to hourly
            df_time['CentralTimeZone'] = df_time['PosixTime'].map(lambda val:datetime.fromtimestamp(val).strftime('%Y-%m-%d %H:%M:%S'))
            df_time['UTCTimeZone'] = pd.to_datetime(df_time['PosixTime'], unit = 's')
            df= pd.concat([df_time,df_M], axis=1, join="inner")
            df = df.set_index(['UTCTimeZone'])
            df_monthly = df.resample('M').mean()  #create monthly datasets
            
            # get vortex ID and attach to project name
            file_list = file.split(".")
            file_id = file_list[3]
            
            # Add data to project_refs dict
            projects[file_id]['data'] = df_monthly
            projects[file_id]['id'] = file_id
            
            # exporting to csv for checks
            # conversion.main()  #not testing with this
            
            # making a list of dicts
            projects_list.append(projects[file_id])
            
             
    return projects_list, projects

In [5]:
def data_processing(data_list):
    """
    processing data to get year over year changes and current month values
    The output of this function is going to look like the file MonthlyWindspeeds_Month_Year from Meghan's email 7/3/2024
    
    inputs:
        data dict imported from netCDF_import function resmapled at monthly values
        
    outputs:
        data dict, including now the monthly averaged values from all previous years, 
        plus the current monthly value for comparison for each month in the dataset (jan-dec)
    """
    month_list = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    
    # pull the latest month in the dataset for comparison for each dataset
    date = datetime.today() + timedelta(-30)  # change back to 30, needs to be 31 for testing on some months but will not matter on the first
    
    # find latest data point for comparison
    for project in data_list:
        project_data = project['data']
        latest = project_data[project_data.index > date]
        
        # in case this is run on the 31st of a month and the previous month does not have a 31st, this will be an empty dataframe
        if len(latest) == 0:
            date = datetime.today() + timedelta(-35)
            latest = project_data[project_data.index > date]
        
        project['latest'] = latest
        
        # average all data points, until i hear otherwise, going to include most recent data point
        project['mean data'] = project['data'].groupby(project['data'].index.strftime('%B'), sort=False).mean()
        
        # groupby for all as well
        project['overall data'] = project['data'].mean()
        
        # adding current year data
        project['current year data'] = project['data'][project['data'].index.year >= date.year]
    
    return data_list

In [6]:
def excel_output(excel_data):
    """
    creating output file with excel
    
    inputs:
        excel formatted data from the processing function 
        
    outputs:
        an excel output file
    """
    # Denmark Deliverable

    wb = Workbook()  # create Workbook object

    month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']

    # repeating myself for a date
    date = datetime.today() + timedelta(-30) # change back to 30 needs to be 31 on some months, will never matter on the first though

    # create exec page
    ws = wb.create_sheet("Result")

    # TODO delete extra sheet
    sheet_to_delete = wb['Sheet']
    wb.remove(sheet_to_delete)


    # Top Table
    b = 1
    c = 2

    #Table Headers
    for month in month_list:
        # write first month and year
        ws.cell(row=b, column=c, value=f"{month} {date.strftime('%Y')}")
        # write first month and all years
        ws.cell(row=b, column=c+1, value=f"{month} All Years")
        c = c+2

        # increment and repeat

    for project in excel_data:
        a = 1
        month_iter = 1
        # project name to row b+1, col a
        ws.cell(row=b+1, column=a, value=project['name'])
        # increment through each month
        for month in month_list:
            # print project latest to row b+1, col a+1
            # determine latest year, find datapoint that is a higher value than that or the previous year for latest one from dataset

            # if for if there is a month matching the current year with month_iter == dataset, if not 0.0 will be value
            try:
                if project['name'] == "Dermott":
                    ws.cell(row=b+1, column=a+1, value=project['current year data']['80'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
                    # print project all years to row b+1, col a+2
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['80'][month_iter - 1])
                elif project['name'] == "Lincoln Land":
                    ws.cell(row=b+1, column=a+1, value=project['current year data']['110'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
                    # print project all years to row b+1, col a+2
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['110'][month_iter - 1])
                else:
                    ws.cell(row=b+1, column=a+1, value=project['current year data']['90'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
                    # print project all years to row b+1, col a+2
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['90'][month_iter - 1])  ##-- if lincoln land or Dermott, pull 110/80 respectively
            except IndexError:
                ws.cell(row=b+1, column=a+1, value=0.0)
                # print project all years to row b+1, col a+2
                if project['name'] == "Dermott":
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['80'][month_iter - 1])
                elif project['name'] == "Lincoln Land":
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['110'][month_iter - 1])
                else:
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['90'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively

            month_iter = month_iter + 1
            a = a+2
        b = b + 1


    # TODO Formatting all cells

    # TODO extend cells width
    for col in range(1, ws.max_column+1):
        column_letter = utils.get_column_letter(col)
        ws.column_dimensions[column_letter].width = 20 # set width for cols used

    style = NamedStyle(name='decimal_style', number_format='0.00')

    #center cell values
    for row in ws.iter_rows():
        for cell in row:
            cell.style = style  # reduce number of decimal places
            cell.alignment = Alignment(horizontal='center', vertical='center')  # center alignment           

    # create deliverable string
    month_str = excel_data[0]['latest'].index.date[0].strftime('%B')
    year_str = excel_data[0]['latest'].index.date[0].strftime('%Y')

    # concat string
    deliverable_string = f"MonthlyWindSpeeds_{month_str}_{year_str}_Onshore.xlsx"

    # save file
    wb.save(f"outputs/{deliverable_string}")
    
    
    return

In [7]:
def excel_internal_output(excel_data):
    """
    creating output file with excel
    
    inputs:
        excel formatted data from the processing function 
        
    outputs:
        an excel output file
    """

    # Internal Deliverable with extra stuff Deliverable
    # TODO remove the top table of last month vs overall dataset

    # Internal Deliverable with extra stuff Deliverable
    # TODO remove the top table of last month vs overall dataset

    wb = Workbook()  # create Workbook object

    month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']

    # repeating myself for a date
    date = datetime.today() + timedelta(-30) # change back to 30 needs to be 31 on some months, will never matter on the first though

    # create deliverable string
    month_str = excel_data[0]['latest'].index.date[0].strftime('%B')
    year_str = excel_data[0]['latest'].index.date[0].strftime('%Y')
    # concat string for deliverable
    deliverable_string = f"MonthlyWindSpeeds_{month_str}_{year_str}_Onshore_Internal.xlsx"

    # create exec page
    ws = wb.create_sheet("Result")

    # TODO delete extra sheet
    sheet_to_delete = wb['Sheet']
    wb.remove(sheet_to_delete)

    # create middle results table first

    # Middle Table

    #Table Headers
    a1 = ws.cell(row=1, column=2, value=f"{date.strftime('%b')} {date.strftime('%Y')}")
    b1 = ws.cell(row=1, column=3, value="All")
    c1 = ws.cell(row=1, column=4, value="HH")
    e1 = ws.cell(row=1, column=5, value="Vortex ID")

    y = 2
    x = 2

    for project in excel_data:
        # project name to row y=2, col x=1
        ws.cell(row=y, column=x-1, value=project['name'])

        # print project latest to row y=2, col x=1  -- if lincoln land or Dermott, pull 110/80 respectively
        if project['name'] == 'Dermott':
            ws.cell(row=y, column=x, value=project['latest']['80'][0])
        elif project['name'] == 'Lincoln Land':
            ws.cell(row=y, column=x, value=project['latest']['110'][0])
        else:
            ws.cell(row=y, column=x, value=project['latest']['90'][0])

        # print project All to row y=2, col x=2 -- if lincoln land or Dermott, pull 110/80 respectively
        if project['name'] == 'Dermott':
            ws.cell(row=y, column=x+1, value=project['overall data']['80'])
        elif project['name'] == 'Lincoln Land':
            ws.cell(row=y, column=x+1, value=project['overall data']['110'])
        else:
            ws.cell(row=y, column=x+1, value=project['overall data']['90'])

        # print project HH to row y=2, col x=3 | could be replaced with a variable if needed -- if lincoln land or Dermott, pull 110/80 respectively
        if project['name'] == 'Dermott':
            ws.cell(row=y, column=x+2, value=80)
        elif project['name'] == "Lincoln Land":
            ws.cell(row=y, column=x+2, value=110)
        else:
            ws.cell(row=y, column=x+2, value=90)

        # print project Vortex ID to row y=2, col x=4
        ws.cell(row=y, column=x+3, value=project['id'])
        # repeat, increment y+1
        y = y+1

    # Top Table
    b = len(excel_data) + 3
    c = 2

    #Table Headers
    for month in month_list:
        # write first month and year
        ws.cell(row=b, column=c, value=f"{month} {date.strftime('%Y')}")
        # write first month and all years
        ws.cell(row=b, column=c+1, value=f"{month} All Years")
        c = c+2

        # increment and repeat

    for project in excel_data:
        a = 1
        month_iter = 1
        # project name to row b+1, col a
        ws.cell(row=b+1, column=a, value=project['name'])
        # increment through each month
        for month in month_list:
            # print project latest to row b+1, col a+1
            # determine latest year, find datapoint that is a higher value than that or the previous year for latest one from dataset

            # if for if there is a month matching the current year with month_iter == dataset, if not 0.0 will be value
            try:
                if project['name'] == "Dermott":
                    ws.cell(row=b+1, column=a+1, value=project['current year data']['80'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
                    # print project all years to row b+1, col a+2
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['80'][month_iter - 1])
                elif project['name'] == "Lincoln Land":
                    ws.cell(row=b+1, column=a+1, value=project['current year data']['110'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
                    # print project all years to row b+1, col a+2
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['110'][month_iter - 1])
                else:
                    ws.cell(row=b+1, column=a+1, value=project['current year data']['90'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
                    # print project all years to row b+1, col a+2
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['90'][month_iter - 1])  ##-- if lincoln land or Dermott, pull 110/80 respectively
            except IndexError:
                ws.cell(row=b+1, column=a+1, value=0.0)
                # print project all years to row b+1, col a+2
                if project['name'] == "Dermott":
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['80'][month_iter - 1])
                elif project['name'] == "Lincoln Land":
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['110'][month_iter - 1])
                else:
                    ws.cell(row=b+1, column=a+2, value=project['mean data']['90'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively

            month_iter = month_iter + 1
            a = a+2
        b = b + 1


    # TODO Add a table 3 with IAV, Yearly Windiness, Yearly Energiness and IAV, similar to Taylor's table


    project_names = []

    # create additional sheets for each project in the dict
    for project in excel_data:
        project_names.append(project['name'])
        # create sheet with project name
        sheet = wb.create_sheet(project['name'])

    # TODO Additional Tables including Windiness, IAV and Energiness - formatted to match Taylor's (As close as possible with the extra data

    for project in excel_data:
        sheet = wb[project['name']]

        for r in dataframe_to_rows(project['data'], index=False, header=True):
            sheet.append(r)


    # save file
    wb.save(f"outputs/{deliverable_string}")
    
    return



In [8]:
def one_month_windiness(project_data_ex):
    """
    Windiness calculation based on the Energiness_Windiness_Manual sheet included in the folder here
    """
    
    #     # TODO MoMMs - Average of each month, from entire dataset, averaged 
    #     # MoMMs
    #     MoMM = project_data['mean data']['90'].mean()

    #     # windiness for each month
    #     windiness = project_data['mean data']['90'] / MoMM

    #     # find the 1 month windiness for every datapoint in the dataframe, perform with an array function preferably
    #     # nov 24 / nov all.mean()
    #     project_data['data'].apply()

    # project_data = data[0]

    # TODO MoMMs - Average of each month, from entire dataset, averaged 
    # MoMMs
    # MoMM = project_data['mean data'].mean()

    # windiness for each month
    # overall_windiness = project_data['mean data'] / MoMM

    # find the 1 month windiness for every datapoint in the dataframe, perform with an array function preferably
    # nov 24 / nov all.mean()
    # filter all of current month to be divided
    # find current month of value

    # project_data_ex[project_data_ex.index.month == 1]


    # New approach - find the individual value in the first row, 
    # get that value's month from the index of the dataframe
    # filter the dataframe with that month
    # mean the filtered dataframe
    # divide the value from the row / mean value of filtered dataframe
    # repeat for every row in the dataframe
    windiness_monthly_calc_list = []

    for index, row in project_data_ex.iterrows():
        #get that value's month from the index of the dataframe
        filter_month = index.month
        # print(index.month)
        # filter the dataframe with that month
        filtered_project_data_ex = project_data_ex[project_data_ex.index.month==filter_month]
        # print(filtered_project_data_ex['90'])
        # mean the filtered dataframe
        filtered_project_data_ex_mean = filtered_project_data_ex['90'].mean()
        # print(filtered_project_data_ex_mean)
        # divide the value from the row / mean value of filtered dataframe
        monthly_windiness_row_90 = row['90'] / filtered_project_data_ex_mean
        # print(monthly_windiness_row_90)
        windiness_monthly_calc_list.append(monthly_windiness_row_90)

    project_data_ex['90 windiness'] = windiness_monthly_calc_list

    project_data_ex
    
    # although this does the trick, this is very slow, will need to adjust how it works in the future
    
    return project_data_ex

In [9]:
def three_month_windiness(project_data_ex):
    """
    three month version of the one month code
    """
    
    # project_data = data[0]

    # MoMMs - Average of each month, from entire dataset, averaged 
    # MoMMs
    # MoMM = project_data['mean data'].mean()

    # windiness for each month
    # overall_windiness = project_data['mean data'] / MoMM

    # find the 1 month windiness for every datapoint in the dataframe, perform with an array function preferably
    # nov 24 / nov all.mean()
    # filter all of current month to be divided
    # find current month of value

    # project_data_ex[project_data_ex.index.month == 1]


    # New approach - find the individual value in the first row, 
    # get that value's month from the index of the dataframe
    # filter the dataframe with that month
    # mean the filtered dataframe
    # divide the value from the row / mean value of filtered dataframe
    # repeat for every row in the dataframe
    windiness_monthly_calc_list = []

    # if both of these vars have a value, the code can run, otherwise, no
    previous_monthly_value_1 = None
    previous_monthly_value_2 = None


    for index, row in project_data_ex.iterrows():

        if previous_monthly_value_1 == None:
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_2 == None:
            # assign previous value
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        else:
            #get that value's month from the index of the dataframe
            filter_month = index.month
            # print(f"filter_month_1 : {filter_month}")
            previous_filter_month_1 = (index - relativedelta(months=1)).month
            # print(f"previous_filter_month_1 : {previous_filter_month_1}")
            previous_filter_month_2 = (index - relativedelta(months=2)).month
            # print(f"previous_filter_month_1 : {previous_filter_month_2}")
            # TODO for three months, there needs to be 3 months, the one before, the current and the one after, plus try statements for the edge cases
            # print(f"{filter_month, previous_filter_month_1, previous_filter_month_2}")

            # TODO do this three times, with all three filtered dataframes, then mean them for our filtered_project_data_ex_mean
            # filter the dataframe with that month
            filtered_project_data_ex_0 = project_data_ex[project_data_ex.index.month==filter_month]
            # mean the filtered dataframe
            filtered_project_data_ex_mean_0 = filtered_project_data_ex_0['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_1 = project_data_ex[project_data_ex.index.month==previous_filter_month_1]
            # mean the filtered dataframe
            filtered_project_data_ex_mean_1 = filtered_project_data_ex_1['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_2 = project_data_ex[project_data_ex.index.month==previous_filter_month_2]
            # mean the filtered dataframe
            filtered_project_data_ex_mean_2 = filtered_project_data_ex_2['90'].mean()

            # meam resulting values
            resulting_monthly_means = [filtered_project_data_ex_mean_0, filtered_project_data_ex_mean_1, filtered_project_data_ex_mean_2]
            overall_df_monthly_mean_of_means = sum(resulting_monthly_means) / len(resulting_monthly_means)
            # print(overall_df_monthly_mean_of_means)


            # divide the value from the row / mean value of filtered dataframe
            # TODO need to get the previous month and the next month and store them in variables to use here. If these variables are empty, skip it and add a missing value in for this calc
            # find mean of current_value, previous_monthly_value_1 and previous_monthly_value_2
            current_monthly_value = row['90']
            numbers = [current_monthly_value, previous_monthly_value_1, previous_monthly_value_2]
            current_monthly_mean = sum(numbers) / len(numbers)

            # three month mean / three month MoMM
            monthly_windiness_row_90_3_month = current_monthly_mean / overall_df_monthly_mean_of_means
            # print(monthly_windiness_row_90_3_month)
            # print(monthly_windiness_row_90_3_month)
            windiness_monthly_calc_list.append(monthly_windiness_row_90_3_month)

            previous_monthly_value_2 = previous_monthly_value_1
            previous_monthly_value_1 = current_monthly_value
            current_monthly_value = None


    project_data_ex['90 windiness 3 month'] = windiness_monthly_calc_list

    project_data_ex

    
    return project_data_ex

In [10]:
def twelve_month_windiness(project_data_ex):
    """
    12 month version of the 3 month function above
    """
    # MoMMs - Average of each month, from entire dataset, averaged 
    # MoMMs
    # MoMM = project_data['mean data'].mean()

    # windiness for each month
    # overall_windiness = project_data['mean data'] / MoMM


    # New approach - find the individual value in the first row, 
    # get that value's month from the index of the dataframe
    # filter the dataframe with that month
    # mean the filtered dataframe
    # divide the value from the row / mean value of filtered dataframe
    # repeat for every row in the dataframe
    windiness_monthly_calc_list = []


    # Make all these for 12 months instead
    # if both of these vars have a value, the code can run, otherwise, no
    previous_monthly_value_1 = None
    previous_monthly_value_2 = None
    previous_monthly_value_3 = None
    previous_monthly_value_4 = None
    previous_monthly_value_5 = None
    previous_monthly_value_6 = None
    previous_monthly_value_7 = None
    previous_monthly_value_8 = None
    previous_monthly_value_9 = None
    previous_monthly_value_10 = None
    previous_monthly_value_11 = None
    previous_monthly_value_12 = None


    for index, row in project_data_ex.iterrows():

        if previous_monthly_value_1 == None:
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_2 == None:
            # assign previous value
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_3 == None:
            # assign previous value
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_4 == None:
            # assign previous value
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_5 == None:
            # assign previous value
            previous_monthly_value_5 = previous_monthly_value_4
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_6 == None:
            # assign previous value
            previous_monthly_value_6 = previous_monthly_value_5
            previous_monthly_value_5 = previous_monthly_value_4
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_7 == None:
            # assign previous value
            previous_monthly_value_7 = previous_monthly_value_6
            previous_monthly_value_6 = previous_monthly_value_5
            previous_monthly_value_5 = previous_monthly_value_4
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_8 == None:
            # assign previous value
            previous_monthly_value_8 = previous_monthly_value_7
            previous_monthly_value_7 = previous_monthly_value_6
            previous_monthly_value_6 = previous_monthly_value_5
            previous_monthly_value_5 = previous_monthly_value_4
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_9 == None:
            # assign previous value
            previous_monthly_value_9 = previous_monthly_value_8
            previous_monthly_value_8 = previous_monthly_value_7
            previous_monthly_value_7 = previous_monthly_value_6
            previous_monthly_value_6 = previous_monthly_value_5
            previous_monthly_value_5 = previous_monthly_value_4
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_10 == None:
            # assign previous value
            previous_monthly_value_10 = previous_monthly_value_9
            previous_monthly_value_9 = previous_monthly_value_8
            previous_monthly_value_8 = previous_monthly_value_7
            previous_monthly_value_7 = previous_monthly_value_6
            previous_monthly_value_6 = previous_monthly_value_5
            previous_monthly_value_5 = previous_monthly_value_4
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_11 == None:
            # assign previous value
            previous_monthly_value_11 = previous_monthly_value_10
            previous_monthly_value_10 = previous_monthly_value_9
            previous_monthly_value_9 = previous_monthly_value_8
            previous_monthly_value_8 = previous_monthly_value_7
            previous_monthly_value_7 = previous_monthly_value_6
            previous_monthly_value_6 = previous_monthly_value_5
            previous_monthly_value_5 = previous_monthly_value_4
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        elif previous_monthly_value_12 == None:
            # assign previous value
            previous_monthly_value_12 = previous_monthly_value_11
            previous_monthly_value_11 = previous_monthly_value_10
            previous_monthly_value_10 = previous_monthly_value_9
            previous_monthly_value_9 = previous_monthly_value_8
            previous_monthly_value_8 = previous_monthly_value_7
            previous_monthly_value_7 = previous_monthly_value_6
            previous_monthly_value_6 = previous_monthly_value_5
            previous_monthly_value_5 = previous_monthly_value_4
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            # assign previous value
            previous_monthly_value_1 = row['90']
            # show missing number for edge case
            windiness_monthly_calc_list.append(None)

        else:
            #get that value's month from the index of the dataframe
            filter_month = index.month
            previous_filter_month_1 = (index - relativedelta(months=1)).month
            previous_filter_month_2 = (index - relativedelta(months=2)).month
            previous_filter_month_3 = (index - relativedelta(months=3)).month
            previous_filter_month_4 = (index - relativedelta(months=4)).month
            previous_filter_month_5 = (index - relativedelta(months=5)).month
            previous_filter_month_6 = (index - relativedelta(months=6)).month
            previous_filter_month_7 = (index - relativedelta(months=7)).month
            previous_filter_month_8 = (index - relativedelta(months=8)).month
            previous_filter_month_9 = (index - relativedelta(months=9)).month
            previous_filter_month_10 = (index - relativedelta(months=10)).month
            previous_filter_month_11 = (index - relativedelta(months=11)).month
            previous_filter_month_12 = (index - relativedelta(months=12)).month

            # TODO do this 12 times, with all three filtered dataframes, then mean them for our filtered_project_data_ex_mean
            # filter the dataframe with that month
            filtered_project_data_ex_0 = project_data_ex[project_data_ex.index.month==filter_month]
            # mean the filtered dataframe
            filtered_project_data_ex_mean_0 = filtered_project_data_ex_0['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_1 = project_data_ex[project_data_ex.index.month==previous_filter_month_1]
            filtered_project_data_ex_mean_1 = filtered_project_data_ex_1['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_2 = project_data_ex[project_data_ex.index.month==previous_filter_month_2]
            filtered_project_data_ex_mean_2 = filtered_project_data_ex_2['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_3 = project_data_ex[project_data_ex.index.month==previous_filter_month_3]
            filtered_project_data_ex_mean_3 = filtered_project_data_ex_3['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_4 = project_data_ex[project_data_ex.index.month==previous_filter_month_4]
            filtered_project_data_ex_mean_4 = filtered_project_data_ex_4['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_5 = project_data_ex[project_data_ex.index.month==previous_filter_month_5]
            filtered_project_data_ex_mean_5 = filtered_project_data_ex_5['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_6 = project_data_ex[project_data_ex.index.month==previous_filter_month_6]
            filtered_project_data_ex_mean_6 = filtered_project_data_ex_6['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_7 = project_data_ex[project_data_ex.index.month==previous_filter_month_7]
            filtered_project_data_ex_mean_7 = filtered_project_data_ex_7['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_8 = project_data_ex[project_data_ex.index.month==previous_filter_month_8]
            filtered_project_data_ex_mean_8 = filtered_project_data_ex_8['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_9 = project_data_ex[project_data_ex.index.month==previous_filter_month_9]
            filtered_project_data_ex_mean_9 = filtered_project_data_ex_9['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_10 = project_data_ex[project_data_ex.index.month==previous_filter_month_10]
            filtered_project_data_ex_mean_10 = filtered_project_data_ex_10['90'].mean()

            # filter the dataframe with that month
            filtered_project_data_ex_11 = project_data_ex[project_data_ex.index.month==previous_filter_month_11]
            filtered_project_data_ex_mean_11 = filtered_project_data_ex_11['90'].mean()

            # meam resulting values
            resulting_monthly_means = [
                filtered_project_data_ex_mean_0, 
                filtered_project_data_ex_mean_1, 
                filtered_project_data_ex_mean_2, 
                filtered_project_data_ex_mean_3, 
                filtered_project_data_ex_mean_4, 
                filtered_project_data_ex_mean_5, 
                filtered_project_data_ex_mean_6, 
                filtered_project_data_ex_mean_7, 
                filtered_project_data_ex_mean_8, 
                filtered_project_data_ex_mean_9, 
                filtered_project_data_ex_mean_10, 
                filtered_project_data_ex_mean_11
            ]
            overall_df_monthly_mean_of_means = sum(resulting_monthly_means) / len(resulting_monthly_means)
            # print(overall_df_monthly_mean_of_means)


            # divide the value from the row / mean value of filtered dataframe
            # TODO need to get the previous month and the next month and store them in variables to use here. If these variables are empty, skip it and add a missing value in for this calc
            # find mean of current_value, previous_monthly_value_1 and previous_monthly_value_2
            current_monthly_value = row['90']
            numbers = [
                current_monthly_value, 
                previous_monthly_value_1, 
                previous_monthly_value_2, 
                previous_monthly_value_3, 
                previous_monthly_value_4, 
                previous_monthly_value_5, 
                previous_monthly_value_6, 
                previous_monthly_value_7, 
                previous_monthly_value_8, 
                previous_monthly_value_9, 
                previous_monthly_value_10, 
                previous_monthly_value_11
            ]
            current_monthly_mean = sum(numbers) / len(numbers)

            # three month mean / three month MoMM
            monthly_windiness_row_90_12_month = current_monthly_mean / overall_df_monthly_mean_of_means
            # print(monthly_windiness_row_90_3_month)
            # print(monthly_windiness_row_90_12_month)
            windiness_monthly_calc_list.append(monthly_windiness_row_90_12_month)

            previous_monthly_value_11 = previous_monthly_value_10
            previous_monthly_value_10 = previous_monthly_value_9
            previous_monthly_value_9 = previous_monthly_value_8
            previous_monthly_value_8 = previous_monthly_value_7
            previous_monthly_value_7 = previous_monthly_value_6
            previous_monthly_value_6 = previous_monthly_value_5
            previous_monthly_value_5 = previous_monthly_value_4
            previous_monthly_value_4 = previous_monthly_value_3
            previous_monthly_value_3 = previous_monthly_value_2
            previous_monthly_value_2 = previous_monthly_value_1
            previous_monthly_value_1 = current_monthly_value
            current_monthly_value = None


    project_data_ex['90 windiness 12 month'] = windiness_monthly_calc_list

    project_data_ex

# although this does the trick, this is very slow, will need to adjust how it works in the future
    
    return project_data_ex

In [11]:
def iav(project_data):
    """
    Interannual variability, mean of each year (jan 01, feb01, mar 01 ... dec 01).mean(), then stdev(all years means)/MoMM of entire dataset
    """
    
    # TODO avg all years
    # IAV
    # TODO MoMMs

    # MoMMs
    MoMM = project_data['mean data'].mean()
    MoMM

    # TODO Mean Years Individually (groupby)
    project_data['data']['timestamp'] = pd.to_datetime(project_data['data'].index)
    project_data['data']['year'] = project_data['data']['timestamp'].dt.year

    # define storage df in projecT_data for IAV values
    project_data['IAV'] = pd.DataFrame()
    
    # IAVs for each metric of windiness
    # TODO StDev all those values
    # divide by MoMMs
    project_data['yearly_avg_wspd'] = project_data['data'].groupby('year')['90'].mean()
    project_data['IAV']['Wind Speed'] = project_data['yearly_avg_wspd'].std() / MoMM
    
    project_data['yearly_avg_windiness'] = project_data['data'].groupby('year')['90 windiness'].mean()
    project_data['IAV']['1 Month Windiness'] = project_data['yearly_avg_windiness'].std() / MoMM
    
    project_data['yearly_avg_windiness 3 month'] = project_data['data'].groupby('year')['90 windiness 3 month'].mean()
    project_data['IAV']['3 Month Windiness'] = project_data['yearly_avg_windiness'].std() / MoMM
    
    project_data['yearly_avg_windiness 12 month'] = project_data['data'].groupby('year')['90 windiness 12 month'].mean()
    project_data['IAV']['12 Month Windiness'] = project_data['yearly_avg_windiness'].std() / MoMM
    
    
    
    return project_data

In [12]:
def energiness(project_data):
    """
    energiness calculation based on the Energiness_Windiness_Manual sheet included in the folder here
    """
    # conversion from the windiness values already in 'data', times the sens value for the project, back to a % out of 100

    # TODO pull all windiness, sens for project
    project_data['data']['90 energiness'] = (project_data['data']['90 windiness'] - 1) * project_data['sens'] + 1

    project_data['data']['90 energiness 3 month'] = (project_data['data']['90 windiness 3 month'] - 1) * project_data['sens'] + 1

    project_data['data']['90 energiness 12 month'] = (project_data['data']['90 windiness 12 month'] - 1) * project_data['sens'] + 1
    
    return project_data
    

In [13]:
data = main()
# data

In [14]:
data[0]

{'name': 'Willow Creek',
 'sens': 1.11,
 'data':                 PosixTime        50        60        70        80        90  \
 UTCTimeZone                                                                   
 2001-01-31   9.796446e+08  7.944086  8.333199  8.694758  9.064516  9.332931   
 2001-02-28   9.821934e+08  6.255506  6.492113  6.710417  6.931399  7.102679   
 2001-03-31   9.847422e+08  6.500269  6.748253  6.973522  7.203495  7.345968   
 2001-04-30   9.873774e+08  7.326250  7.572083  7.798056  8.022639  8.187361   
 2001-05-31   9.900126e+08  7.154570  7.418414  7.653495  7.893414  8.054436   
 ...                   ...       ...       ...       ...       ...       ...   
 2024-08-31   1.723808e+09  6.332393  6.573387  6.790188  7.018548  7.172446   
 2024-09-30   1.726443e+09  6.890833  7.209722  7.507500  7.810417  8.010834   
 2024-10-31   1.729078e+09  6.466667  6.746102  6.996774  7.253360  7.410887   
 2024-11-30   1.731713e+09  7.087083  7.361667  7.612778  7.873194  8.05

In [15]:
data[3]['name'] # use this to check data, this is the mean values for each month for the whole dataset

'Dermott'

In [16]:
project_data = data[0]

# MoMMs - Average of each month, from entire dataset, averaged 
# MoMMs
# MoMM = project_data['mean data'].mean()

# # windiness for each month
# overall_windiness = project_data['mean data'] / MoMM

# # find the 1 month windiness for every datapoint in the dataframe, perform with an array function preferably
# # nov 24 / nov all.mean()
# # filter all of current month to be divided
# # find current month of value
# project_data_ex = project_data['data']

# # project_data_ex[project_data_ex.index.month == 1]


# # New approach - find the individual value in the first row, 
# # get that value's month from the index of the dataframe
# # filter the dataframe with that month
# # mean the filtered dataframe
# # divide the value from the row / mean value of filtered dataframe
# # repeat for every row in the dataframe
# windiness_monthly_calc_list = []


# # TODO Make all these for 12 months instead
# # if both of these vars have a value, the code can run, otherwise, no
# previous_monthly_value_1 = None
# previous_monthly_value_2 = None
# previous_monthly_value_3 = None
# previous_monthly_value_4 = None
# previous_monthly_value_5 = None
# previous_monthly_value_6 = None
# previous_monthly_value_7 = None
# previous_monthly_value_8 = None
# previous_monthly_value_9 = None
# previous_monthly_value_10 = None
# previous_monthly_value_11 = None
# previous_monthly_value_12 = None


# for index, row in project_data_ex.iterrows():
    
#     if previous_monthly_value_1 == None:
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     elif previous_monthly_value_2 == None:
#         # assign previous value
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     elif previous_monthly_value_3 == None:
#         # assign previous value
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
    
#     elif previous_monthly_value_4 == None:
#         # assign previous value
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     elif previous_monthly_value_5 == None:
#         # assign previous value
#         previous_monthly_value_5 = previous_monthly_value_4
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     elif previous_monthly_value_6 == None:
#         # assign previous value
#         previous_monthly_value_6 = previous_monthly_value_5
#         previous_monthly_value_5 = previous_monthly_value_4
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
    
#     elif previous_monthly_value_7 == None:
#         # assign previous value
#         previous_monthly_value_7 = previous_monthly_value_6
#         previous_monthly_value_6 = previous_monthly_value_5
#         previous_monthly_value_5 = previous_monthly_value_4
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     elif previous_monthly_value_8 == None:
#         # assign previous value
#         previous_monthly_value_8 = previous_monthly_value_7
#         previous_monthly_value_7 = previous_monthly_value_6
#         previous_monthly_value_6 = previous_monthly_value_5
#         previous_monthly_value_5 = previous_monthly_value_4
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     elif previous_monthly_value_9 == None:
#         # assign previous value
#         previous_monthly_value_9 = previous_monthly_value_8
#         previous_monthly_value_8 = previous_monthly_value_7
#         previous_monthly_value_7 = previous_monthly_value_6
#         previous_monthly_value_6 = previous_monthly_value_5
#         previous_monthly_value_5 = previous_monthly_value_4
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     elif previous_monthly_value_10 == None:
#         # assign previous value
#         previous_monthly_value_10 = previous_monthly_value_9
#         previous_monthly_value_9 = previous_monthly_value_8
#         previous_monthly_value_8 = previous_monthly_value_7
#         previous_monthly_value_7 = previous_monthly_value_6
#         previous_monthly_value_6 = previous_monthly_value_5
#         previous_monthly_value_5 = previous_monthly_value_4
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     elif previous_monthly_value_11 == None:
#         # assign previous value
#         previous_monthly_value_11 = previous_monthly_value_10
#         previous_monthly_value_10 = previous_monthly_value_9
#         previous_monthly_value_9 = previous_monthly_value_8
#         previous_monthly_value_8 = previous_monthly_value_7
#         previous_monthly_value_7 = previous_monthly_value_6
#         previous_monthly_value_6 = previous_monthly_value_5
#         previous_monthly_value_5 = previous_monthly_value_4
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     elif previous_monthly_value_12 == None:
#         # assign previous value
#         previous_monthly_value_12 = previous_monthly_value_11
#         previous_monthly_value_11 = previous_monthly_value_10
#         previous_monthly_value_10 = previous_monthly_value_9
#         previous_monthly_value_9 = previous_monthly_value_8
#         previous_monthly_value_8 = previous_monthly_value_7
#         previous_monthly_value_7 = previous_monthly_value_6
#         previous_monthly_value_6 = previous_monthly_value_5
#         previous_monthly_value_5 = previous_monthly_value_4
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         # assign previous value
#         previous_monthly_value_1 = row['90']
#         # show missing number for edge case
#         windiness_monthly_calc_list.append(None)
        
#     else:
#         #get that value's month from the index of the dataframe
#         filter_month = index.month
#         previous_filter_month_1 = (index - relativedelta(months=1)).month
#         previous_filter_month_2 = (index - relativedelta(months=2)).month
#         previous_filter_month_3 = (index - relativedelta(months=3)).month
#         previous_filter_month_4 = (index - relativedelta(months=4)).month
#         previous_filter_month_5 = (index - relativedelta(months=5)).month
#         previous_filter_month_6 = (index - relativedelta(months=6)).month
#         previous_filter_month_7 = (index - relativedelta(months=7)).month
#         previous_filter_month_8 = (index - relativedelta(months=8)).month
#         previous_filter_month_9 = (index - relativedelta(months=9)).month
#         previous_filter_month_10 = (index - relativedelta(months=10)).month
#         previous_filter_month_11 = (index - relativedelta(months=11)).month
#         previous_filter_month_12 = (index - relativedelta(months=12)).month
        
#         # TODO do this 12 times, with all three filtered dataframes, then mean them for our filtered_project_data_ex_mean
#         # filter the dataframe with that month
#         filtered_project_data_ex_0 = project_data_ex[project_data_ex.index.month==filter_month]
#         # mean the filtered dataframe
#         filtered_project_data_ex_mean_0 = filtered_project_data_ex_0['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_1 = project_data_ex[project_data_ex.index.month==previous_filter_month_1]
#         filtered_project_data_ex_mean_1 = filtered_project_data_ex_1['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_2 = project_data_ex[project_data_ex.index.month==previous_filter_month_2]
#         filtered_project_data_ex_mean_2 = filtered_project_data_ex_2['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_3 = project_data_ex[project_data_ex.index.month==previous_filter_month_3]
#         filtered_project_data_ex_mean_3 = filtered_project_data_ex_3['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_4 = project_data_ex[project_data_ex.index.month==previous_filter_month_4]
#         filtered_project_data_ex_mean_4 = filtered_project_data_ex_4['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_5 = project_data_ex[project_data_ex.index.month==previous_filter_month_5]
#         filtered_project_data_ex_mean_5 = filtered_project_data_ex_5['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_6 = project_data_ex[project_data_ex.index.month==previous_filter_month_6]
#         filtered_project_data_ex_mean_6 = filtered_project_data_ex_6['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_7 = project_data_ex[project_data_ex.index.month==previous_filter_month_7]
#         filtered_project_data_ex_mean_7 = filtered_project_data_ex_7['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_8 = project_data_ex[project_data_ex.index.month==previous_filter_month_8]
#         filtered_project_data_ex_mean_8 = filtered_project_data_ex_8['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_9 = project_data_ex[project_data_ex.index.month==previous_filter_month_9]
#         filtered_project_data_ex_mean_9 = filtered_project_data_ex_9['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_10 = project_data_ex[project_data_ex.index.month==previous_filter_month_10]
#         filtered_project_data_ex_mean_10 = filtered_project_data_ex_10['90'].mean()
        
#         # filter the dataframe with that month
#         filtered_project_data_ex_11 = project_data_ex[project_data_ex.index.month==previous_filter_month_11]
#         filtered_project_data_ex_mean_11 = filtered_project_data_ex_11['90'].mean()
        
#         # meam resulting values
#         resulting_monthly_means = [
#             filtered_project_data_ex_mean_0, 
#             filtered_project_data_ex_mean_1, 
#             filtered_project_data_ex_mean_2, 
#             filtered_project_data_ex_mean_3, 
#             filtered_project_data_ex_mean_4, 
#             filtered_project_data_ex_mean_5, 
#             filtered_project_data_ex_mean_6, 
#             filtered_project_data_ex_mean_7, 
#             filtered_project_data_ex_mean_8, 
#             filtered_project_data_ex_mean_9, 
#             filtered_project_data_ex_mean_10, 
#             filtered_project_data_ex_mean_11
#         ]
#         overall_df_monthly_mean_of_means = sum(resulting_monthly_means) / len(resulting_monthly_means)
#         # print(overall_df_monthly_mean_of_means)


#         # divide the value from the row / mean value of filtered dataframe
#         # TODO need to get the previous month and the next month and store them in variables to use here. If these variables are empty, skip it and add a missing value in for this calc
#         # find mean of current_value, previous_monthly_value_1 and previous_monthly_value_2
#         current_monthly_value = row['90']
#         numbers = [
#             current_monthly_value, 
#             previous_monthly_value_1, 
#             previous_monthly_value_2, 
#             previous_monthly_value_3, 
#             previous_monthly_value_4, 
#             previous_monthly_value_5, 
#             previous_monthly_value_6, 
#             previous_monthly_value_7, 
#             previous_monthly_value_8, 
#             previous_monthly_value_9, 
#             previous_monthly_value_10, 
#             previous_monthly_value_11
#         ]
#         current_monthly_mean = sum(numbers) / len(numbers)

#         # three month mean / three month MoMM
#         monthly_windiness_row_90_12_month = current_monthly_mean / overall_df_monthly_mean_of_means
#         # print(monthly_windiness_row_90_3_month)
#         # print(monthly_windiness_row_90_12_month)
#         windiness_monthly_calc_list.append(monthly_windiness_row_90_12_month)
        
#         previous_monthly_value_11 = previous_monthly_value_10
#         previous_monthly_value_10 = previous_monthly_value_9
#         previous_monthly_value_9 = previous_monthly_value_8
#         previous_monthly_value_8 = previous_monthly_value_7
#         previous_monthly_value_7 = previous_monthly_value_6
#         previous_monthly_value_6 = previous_monthly_value_5
#         previous_monthly_value_5 = previous_monthly_value_4
#         previous_monthly_value_4 = previous_monthly_value_3
#         previous_monthly_value_3 = previous_monthly_value_2
#         previous_monthly_value_2 = previous_monthly_value_1
#         previous_monthly_value_1 = current_monthly_value
#         current_monthly_value = None
        
    
# project_data_ex['90 windiness 12 month'] = windiness_monthly_calc_list
    
# project_data_ex

# # although this does the trick, this is very slow, will need to adjust how it works in the future




In [17]:
# # IAV

# project_data = data[0]
# project_data

# # TODO MoMMs

# # MoMMs
# MoMM = project_data['mean data'].mean()
# MoMM

# # TODO Mean Years Individually (groupby)
# project_data['data']['timestamp'] = pd.to_datetime(project_data['data'].index)

# project_data['data']['year'] = project_data['data']['timestamp'].dt.year

# # IAVs for each metric
# project_data['yearly_avg_wspd'] = project_data['data'].groupby('year')['90'].mean()
# project_data['yearly_avg_windiness'] = project_data['data'].groupby('year')['90 windiness'].mean()

# # TODO StDev all those values
# # divide by MoMMs
# project_data['IAV']['Wind Speed'] = project_data['yearly_avg_wspd'].std() / MoMM
# project_data['IAV']['1 Month Windiness'] = project_data['yearly_avg_windiness'].std() / MoMM


In [18]:
# # energiness
# project_data = data[0]
# project_data
# # conversion from the windiness values already in 'data', times the sens value for the project, back to a % out of 100

# # TODO pull all windiness, sens for project
# project_data['data']['90 energiness'] = (project_data['data']['90 windiness'] - 1) * project_data['sens'] + 1

# project_data['data']['90 energiness 3 month'] = (project_data['data']['90 windiness 3 month'] - 1) * project_data['sens'] + 1

# project_data['data']['90 energiness 12 month'] = (project_data['data']['90 windiness 12 month'] - 1) * project_data['sens'] + 1

# project_data['data']
# # TODO subtract 1 from windiness value

# # TODO mult by sens value for project

# # TODO add 1 back to value for energiness

In [19]:
data[0]['latest'].index.date[0].strftime('%Y')

'2024'

In [20]:

# # included for testing in other cell
# excel_data = data
    
# # Denmark Deliverable
# # TODO remove the top table of last month vs overall dataset

# wb = Workbook()  # create Workbook object

# month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
#       'August', 'September', 'October', 'November', 'December']

# # repeating myself for a date
# date = datetime.today() + timedelta(-30) # change back to 30 needs to be 31 on some months, will never matter on the first though

# # create exec page
# ws = wb.create_sheet("Result")

# # TODO delete extra sheet
# sheet_to_delete = wb['Sheet']
# wb.remove(sheet_to_delete)


# # Top Table
# b = 1
# c = 2

# #Table Headers
# for month in month_list:
#     # write first month and year
#     ws.cell(row=b, column=c, value=f"{month} {date.strftime('%Y')}")
#     # write first month and all years
#     ws.cell(row=b, column=c+1, value=f"{month} All Years")
#     c = c+2

#     # increment and repeat

# for project in excel_data:
#     a = 1
#     month_iter = 1
#     # project name to row b+1, col a
#     ws.cell(row=b+1, column=a, value=project['name'])
#     # increment through each month
#     for month in month_list:
#         # print project latest to row b+1, col a+1
#         # determine latest year, find datapoint that is a higher value than that or the previous year for latest one from dataset

#         # if for if there is a month matching the current year with month_iter == dataset, if not 0.0 will be value
#         try:
#             if project['name'] == "Dermott":
#                 ws.cell(row=b+1, column=a+1, value=project['current year data']['80'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
#                 # print project all years to row b+1, col a+2
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['80'][month_iter - 1])
#             elif project['name'] == "Lincoln Land":
#                 ws.cell(row=b+1, column=a+1, value=project['current year data']['110'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
#                 # print project all years to row b+1, col a+2
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['110'][month_iter - 1])
#             else:
#                 ws.cell(row=b+1, column=a+1, value=project['current year data']['90'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
#                 # print project all years to row b+1, col a+2
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['90'][month_iter - 1])  ##-- if lincoln land or Dermott, pull 110/80 respectively
#         except IndexError:
#             ws.cell(row=b+1, column=a+1, value=0.0)
#             # print project all years to row b+1, col a+2
#             if project['name'] == "Dermott":
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['80'][month_iter - 1])
#             elif project['name'] == "Lincoln Land":
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['110'][month_iter - 1])
#             else:
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['90'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively

#         month_iter = month_iter + 1
#         a = a+2
#     b = b + 1

    
# # TODO Formatting all cells

# # TODO extend cells width
# for col in range(1, ws.max_column+1):
#     column_letter = utils.get_column_letter(col)
#     ws.column_dimensions[column_letter].width = 20 # set width for cols used

# style = NamedStyle(name='decimal_style', number_format='0.00')
    
# #center cell values
# for row in ws.iter_rows():
#     for cell in row:
#         cell.style = style  # reduce number of decimal places
#         cell.alignment = Alignment(horizontal='center', vertical='center')  # center alignment           
        
# # create deliverable string
# month_str = excel_data[0]['latest'].index.date[0].strftime('%B')
# year_str = excel_data[0]['latest'].index.date[0].strftime('%Y')

# # concat string
# deliverable_string = f"MonthlyWindSpeeds_{month_str}_{year_str}_Onshore.xlsx"

# # save file
# wb.save(f"outputs/{deliverable_string}")

In [21]:
data[0]['data']

,PosixTime,50,60,70,80,90,100,110,120,130,...,280,300,90 windiness,90 windiness 3 month,90 windiness 12 month,90 energiness,90 energiness 3 month,90 energiness 12 month,timestamp,year
UTCTimeZone,,,,,,,,,,,,,,,,,,,,,
2001-01-31,9.796446e+08,7.944086,8.333199,8.694758,9.064516,9.332931,9.611424,9.896505,10.078629,10.253763,...,11.560215,11.584543,1.053506,NaN,NaN,1.059391,NaN,NaN,2001-01-31,2001
2001-02-28,9.821934e+08,6.255506,6.492113,6.710417,6.931399,7.102679,7.278572,7.458333,7.580506,7.700149,...,8.637797,8.669642,0.884301,NaN,NaN,0.871574,NaN,NaN,2001-02-28,2001
2001-03-31,9.847422e+08,6.500269,6.748253,6.973522,7.203495,7.345968,7.497178,7.654570,7.742339,7.823791,...,7.973656,7.919892,0.901905,0.949901,NaN,0.891115,0.944390,NaN,2001-03-31,2001
2001-04-30,9.873774e+08,7.326250,7.572083,7.798056,8.022639,8.187361,8.353889,8.524861,8.636528,8.745694,...,9.486944,9.515833,0.983612,0.923893,NaN,0.981809,0.915521,NaN,2001-04-30,2001
2001-05-31,9.900126e+08,7.154570,7.418414,7.653495,7.893414,8.054436,8.217339,8.385080,8.488844,8.589382,...,9.338037,9.366398,1.011335,0.965411,NaN,1.012582,0.961606,NaN,2001-05-31,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,1.723808e+09,6.332393,6.573387,6.790188,7.018548,7.172446,7.334274,7.502016,7.610484,7.715322,...,8.380779,8.376344,1.015126,1.000843,0.983069,1.016790,1.000935,0.981207,2024-08-31,2024
2024-09-30,1.726443e+09,6.890833,7.209722,7.507500,7.810417,8.010834,8.217361,8.430695,8.554028,8.668056,...,9.323195,9.333889,1.073637,1.003019,0.993645,1.081738,1.003352,0.992946,2024-09-30,2024
2024-10-31,1.729078e+09,6.466667,6.746102,6.996774,7.253360,7.410887,7.575538,7.745430,7.837635,7.924597,...,8.256586,8.234139,0.896099,0.991097,0.988224,0.884670,0.990118,0.986929,2024-10-31,2024


In [22]:
# # included for testing in other cell
# from openpyxl.utils.dataframe import dataframe_to_rows
# excel_data = data
    
# # Internal Deliverable with extra stuff Deliverable
# # TODO remove the top table of last month vs overall dataset

# wb = Workbook()  # create Workbook object

# month_list = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
#       'August', 'September', 'October', 'November', 'December']

# # repeating myself for a date
# date = datetime.today() + timedelta(-30) # change back to 30 needs to be 31 on some months, will never matter on the first though

# # create deliverable string
# month_str = excel_data[0]['latest'].index.date[0].strftime('%B')
# year_str = excel_data[0]['latest'].index.date[0].strftime('%Y')

# # concat string for deliverable
# deliverable_string = f"MonthlyWindSpeeds_{month_str}_{year_str}_Onshore_Internal.xlsx"

# # create exec page
# ws = wb.create_sheet("Result")

# # TODO delete extra sheet
# sheet_to_delete = wb['Sheet']
# wb.remove(sheet_to_delete)

# # create middle results table first

# # Middle Table

# #Table Headers
# a1 = ws.cell(row=1, column=2, value=f"{date.strftime('%b')} {date.strftime('%Y')}")
# b1 = ws.cell(row=1, column=3, value="All")
# c1 = ws.cell(row=1, column=4, value="HH")
# e1 = ws.cell(row=1, column=5, value="Vortex ID")

# y = 2
# x = 2

# for project in excel_data:
#     # project name to row y=2, col x=1
#     ws.cell(row=y, column=x-1, value=project['name'])

#     # print project latest to row y=2, col x=1  -- if lincoln land or Dermott, pull 110/80 respectively
#     if project['name'] == 'Dermott':
#         ws.cell(row=y, column=x, value=project['latest']['80'][0])
#     elif project['name'] == 'Lincoln Land':
#         ws.cell(row=y, column=x, value=project['latest']['110'][0])
#     else:
#         ws.cell(row=y, column=x, value=project['latest']['90'][0])

#     # print project All to row y=2, col x=2 -- if lincoln land or Dermott, pull 110/80 respectively
#     if project['name'] == 'Dermott':
#         ws.cell(row=y, column=x+1, value=project['overall data']['80'])
#     elif project['name'] == 'Lincoln Land':
#         ws.cell(row=y, column=x+1, value=project['overall data']['110'])
#     else:
#         ws.cell(row=y, column=x+1, value=project['overall data']['90'])

#     # print project HH to row y=2, col x=3 | could be replaced with a variable if needed -- if lincoln land or Dermott, pull 110/80 respectively
#     if project['name'] == 'Dermott':
#         ws.cell(row=y, column=x+2, value=80)
#     elif project['name'] == "Lincoln Land":
#         ws.cell(row=y, column=x+2, value=110)
#     else:
#         ws.cell(row=y, column=x+2, value=90)

#     # print project Vortex ID to row y=2, col x=4
#     ws.cell(row=y, column=x+3, value=project['id'])
#     # repeat, increment y+1
#     y = y+1

# # Top Table
# b = len(excel_data) + 3
# c = 2

# #Table Headers
# for month in month_list:
#     # write first month and year
#     ws.cell(row=b, column=c, value=f"{month} {date.strftime('%Y')}")
#     # write first month and all years
#     ws.cell(row=b, column=c+1, value=f"{month} All Years")
#     c = c+2

#     # increment and repeat

# for project in excel_data:
#     a = 1
#     month_iter = 1
#     # project name to row b+1, col a
#     ws.cell(row=b+1, column=a, value=project['name'])
#     # increment through each month
#     for month in month_list:
#         # print project latest to row b+1, col a+1
#         # determine latest year, find datapoint that is a higher value than that or the previous year for latest one from dataset

#         # if for if there is a month matching the current year with month_iter == dataset, if not 0.0 will be value
#         try:
#             if project['name'] == "Dermott":
#                 ws.cell(row=b+1, column=a+1, value=project['current year data']['80'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
#                 # print project all years to row b+1, col a+2
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['80'][month_iter - 1])
#             elif project['name'] == "Lincoln Land":
#                 ws.cell(row=b+1, column=a+1, value=project['current year data']['110'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
#                 # print project all years to row b+1, col a+2
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['110'][month_iter - 1])
#             else:
#                 ws.cell(row=b+1, column=a+1, value=project['current year data']['90'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively
#                 # print project all years to row b+1, col a+2
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['90'][month_iter - 1])  ##-- if lincoln land or Dermott, pull 110/80 respectively
#         except IndexError:
#             ws.cell(row=b+1, column=a+1, value=0.0)
#             # print project all years to row b+1, col a+2
#             if project['name'] == "Dermott":
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['80'][month_iter - 1])
#             elif project['name'] == "Lincoln Land":
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['110'][month_iter - 1])
#             else:
#                 ws.cell(row=b+1, column=a+2, value=project['mean data']['90'][month_iter - 1])  ## -- if lincoln land or Dermott, pull 110/80 respectively

#         month_iter = month_iter + 1
#         a = a+2
#     b = b + 1

    
# # TODO Add a table 3 with IAV, Yearly Windiness, Yearly Energiness and IAV, similar to Taylor's table
    
    
# project_names = []

# # create additional sheets for each project in the dict
# for project in excel_data:
#     project_names.append(project['name'])
#     # create sheet with project name
#     sheet = wb.create_sheet(project['name'])
    
# # TODO Additional Tables including Windiness, IAV and Energiness - formatted to match Taylor's (As close as possible with the extra data

# for project in excel_data:
#     sheet = wb[project['name']]

#     for r in dataframe_to_rows(project['data'], index=False, header=True):
#         sheet.append(r)


# # save file
# wb.save(f"outputs/{deliverable_string}")

In [23]:
# def twelve_month_windiness(project_data_ex):
project_data_ex = data[0]['data']
"""
12 month version of the 3 month function above
"""
# MoMMs - Average of each month, from entire dataset, averaged 
# MoMMs
# MoMM = project_data['mean data'].mean()

# windiness for each month
# overall_windiness = project_data['mean data'] / MoMM


# New approach - find the individual value in the first row, 
# get that value's month from the index of the dataframe
# filter the dataframe with that month
# mean the filtered dataframe
# divide the value from the row / mean value of filtered dataframe
# repeat for every row in the dataframe
windiness_monthly_calc_list = []


# Make all these for 12 months instead
# if both of these vars have a value, the code can run, otherwise, no
previous_monthly_value_1 = None
previous_monthly_value_2 = None
previous_monthly_value_3 = None
previous_monthly_value_4 = None
previous_monthly_value_5 = None
previous_monthly_value_6 = None
previous_monthly_value_7 = None
previous_monthly_value_8 = None
previous_monthly_value_9 = None
previous_monthly_value_10 = None
previous_monthly_value_11 = None
previous_monthly_value_12 = None


for index, row in project_data_ex.iterrows():

    if previous_monthly_value_1 == None:
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_2 == None:
        # assign previous value
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_3 == None:
        # assign previous value
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_4 == None:
        # assign previous value
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_5 == None:
        # assign previous value
        previous_monthly_value_5 = previous_monthly_value_4
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_6 == None:
        # assign previous value
        previous_monthly_value_6 = previous_monthly_value_5
        previous_monthly_value_5 = previous_monthly_value_4
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_7 == None:
        # assign previous value
        previous_monthly_value_7 = previous_monthly_value_6
        previous_monthly_value_6 = previous_monthly_value_5
        previous_monthly_value_5 = previous_monthly_value_4
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_8 == None:
        # assign previous value
        previous_monthly_value_8 = previous_monthly_value_7
        previous_monthly_value_7 = previous_monthly_value_6
        previous_monthly_value_6 = previous_monthly_value_5
        previous_monthly_value_5 = previous_monthly_value_4
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_9 == None:
        # assign previous value
        previous_monthly_value_9 = previous_monthly_value_8
        previous_monthly_value_8 = previous_monthly_value_7
        previous_monthly_value_7 = previous_monthly_value_6
        previous_monthly_value_6 = previous_monthly_value_5
        previous_monthly_value_5 = previous_monthly_value_4
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_10 == None:
        # assign previous value
        previous_monthly_value_10 = previous_monthly_value_9
        previous_monthly_value_9 = previous_monthly_value_8
        previous_monthly_value_8 = previous_monthly_value_7
        previous_monthly_value_7 = previous_monthly_value_6
        previous_monthly_value_6 = previous_monthly_value_5
        previous_monthly_value_5 = previous_monthly_value_4
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_11 == None:
        # assign previous value
        previous_monthly_value_11 = previous_monthly_value_10
        previous_monthly_value_10 = previous_monthly_value_9
        previous_monthly_value_9 = previous_monthly_value_8
        previous_monthly_value_8 = previous_monthly_value_7
        previous_monthly_value_7 = previous_monthly_value_6
        previous_monthly_value_6 = previous_monthly_value_5
        previous_monthly_value_5 = previous_monthly_value_4
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    elif previous_monthly_value_12 == None:
        # assign previous value
        previous_monthly_value_12 = previous_monthly_value_11
        previous_monthly_value_11 = previous_monthly_value_10
        previous_monthly_value_10 = previous_monthly_value_9
        previous_monthly_value_9 = previous_monthly_value_8
        previous_monthly_value_8 = previous_monthly_value_7
        previous_monthly_value_7 = previous_monthly_value_6
        previous_monthly_value_6 = previous_monthly_value_5
        previous_monthly_value_5 = previous_monthly_value_4
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        # assign previous value
        previous_monthly_value_1 = row['90']
        # show missing number for edge case
        windiness_monthly_calc_list.append(None)

    else:
        #get that value's month from the index of the dataframe
        filter_month = index.month
        previous_filter_month_1 = (index - relativedelta(months=1)).month
        previous_filter_month_2 = (index - relativedelta(months=2)).month
        previous_filter_month_3 = (index - relativedelta(months=3)).month
        previous_filter_month_4 = (index - relativedelta(months=4)).month
        previous_filter_month_5 = (index - relativedelta(months=5)).month
        previous_filter_month_6 = (index - relativedelta(months=6)).month
        previous_filter_month_7 = (index - relativedelta(months=7)).month
        previous_filter_month_8 = (index - relativedelta(months=8)).month
        previous_filter_month_9 = (index - relativedelta(months=9)).month
        previous_filter_month_10 = (index - relativedelta(months=10)).month
        previous_filter_month_11 = (index - relativedelta(months=11)).month
        previous_filter_month_12 = (index - relativedelta(months=12)).month

        # TODO do this 12 times, with all three filtered dataframes, then mean them for our filtered_project_data_ex_mean
        # filter the dataframe with that month
        filtered_project_data_ex_0 = project_data_ex[project_data_ex.index.month==filter_month]
        # mean the filtered dataframe
        filtered_project_data_ex_mean_0 = filtered_project_data_ex_0['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_1 = project_data_ex[project_data_ex.index.month==previous_filter_month_1]
        filtered_project_data_ex_mean_1 = filtered_project_data_ex_1['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_2 = project_data_ex[project_data_ex.index.month==previous_filter_month_2]
        filtered_project_data_ex_mean_2 = filtered_project_data_ex_2['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_3 = project_data_ex[project_data_ex.index.month==previous_filter_month_3]
        filtered_project_data_ex_mean_3 = filtered_project_data_ex_3['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_4 = project_data_ex[project_data_ex.index.month==previous_filter_month_4]
        filtered_project_data_ex_mean_4 = filtered_project_data_ex_4['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_5 = project_data_ex[project_data_ex.index.month==previous_filter_month_5]
        filtered_project_data_ex_mean_5 = filtered_project_data_ex_5['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_6 = project_data_ex[project_data_ex.index.month==previous_filter_month_6]
        filtered_project_data_ex_mean_6 = filtered_project_data_ex_6['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_7 = project_data_ex[project_data_ex.index.month==previous_filter_month_7]
        filtered_project_data_ex_mean_7 = filtered_project_data_ex_7['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_8 = project_data_ex[project_data_ex.index.month==previous_filter_month_8]
        filtered_project_data_ex_mean_8 = filtered_project_data_ex_8['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_9 = project_data_ex[project_data_ex.index.month==previous_filter_month_9]
        filtered_project_data_ex_mean_9 = filtered_project_data_ex_9['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_10 = project_data_ex[project_data_ex.index.month==previous_filter_month_10]
        filtered_project_data_ex_mean_10 = filtered_project_data_ex_10['90'].mean()

        # filter the dataframe with that month
        filtered_project_data_ex_11 = project_data_ex[project_data_ex.index.month==previous_filter_month_11]
        filtered_project_data_ex_mean_11 = filtered_project_data_ex_11['90'].mean()

        # meam resulting values
        resulting_monthly_means = [
            filtered_project_data_ex_mean_0, 
            filtered_project_data_ex_mean_1, 
            filtered_project_data_ex_mean_2, 
            filtered_project_data_ex_mean_3, 
            filtered_project_data_ex_mean_4, 
            filtered_project_data_ex_mean_5, 
            filtered_project_data_ex_mean_6, 
            filtered_project_data_ex_mean_7, 
            filtered_project_data_ex_mean_8, 
            filtered_project_data_ex_mean_9, 
            filtered_project_data_ex_mean_10, 
            filtered_project_data_ex_mean_11
        ]
        
        
        overall_df_monthly_mean_of_means = sum(resulting_monthly_means) / len(resulting_monthly_means)
        # print(overall_df_monthly_mean_of_means)
            
        # divide the value from the row / mean value of filtered dataframe
        # TODO need to get the previous month and the next month and store them in variables to use here. If these variables are empty, skip it and add a missing value in for this calc
        # find mean of current_value, previous_monthly_value_1 and previous_monthly_value_2
        current_monthly_value = row['90']
        numbers = [
            current_monthly_value, 
            previous_monthly_value_1, 
            previous_monthly_value_2, 
            previous_monthly_value_3, 
            previous_monthly_value_4, 
            previous_monthly_value_5, 
            previous_monthly_value_6, 
            previous_monthly_value_7, 
            previous_monthly_value_8, 
            previous_monthly_value_9, 
            previous_monthly_value_10, 
            previous_monthly_value_11
        ]
        print(numbers)
        current_monthly_mean = sum(numbers) / len(numbers)

        # three month mean / three month MoMM
        monthly_windiness_row_90_12_month = current_monthly_mean / overall_df_monthly_mean_of_means
        # print(monthly_windiness_row_90_3_month)
        # print(monthly_windiness_row_90_12_month)
        windiness_monthly_calc_list.append(monthly_windiness_row_90_12_month)

        previous_monthly_value_11 = previous_monthly_value_10
        previous_monthly_value_10 = previous_monthly_value_9
        previous_monthly_value_9 = previous_monthly_value_8
        previous_monthly_value_8 = previous_monthly_value_7
        previous_monthly_value_7 = previous_monthly_value_6
        previous_monthly_value_6 = previous_monthly_value_5
        previous_monthly_value_5 = previous_monthly_value_4
        previous_monthly_value_4 = previous_monthly_value_3
        previous_monthly_value_3 = previous_monthly_value_2
        previous_monthly_value_2 = previous_monthly_value_1
        previous_monthly_value_1 = current_monthly_value
        current_monthly_value = None


project_data_ex['90 windiness 12 month'] = windiness_monthly_calc_list

project_data_ex

# although this does the trick, this is very slow, will need to adjust how it works in the future
    
# return project_data_ex

[9.09704303741455, 8.927285194396973, 6.683888912200928, 8.306451797485352, 7.735138893127441, 7.442338943481445, 6.886693477630615, 7.515555381774902, 8.054435729980469, 8.187360763549805, 7.345967769622803, 7.1026787757873535]
[8.84375, 9.09704303741455, 8.927285194396973, 6.683888912200928, 8.306451797485352, 7.735138893127441, 7.442338943481445, 6.886693477630615, 7.515555381774902, 8.054435729980469, 8.187360763549805, 7.345967769622803]
[7.7229838371276855, 8.84375, 9.09704303741455, 8.927285194396973, 6.683888912200928, 8.306451797485352, 7.735138893127441, 7.442338943481445, 6.886693477630615, 7.515555381774902, 8.054435729980469, 8.187360763549805]
[7.615833282470703, 7.7229838371276855, 8.84375, 9.09704303741455, 8.927285194396973, 6.683888912200928, 8.306451797485352, 7.735138893127441, 7.442338943481445, 6.886693477630615, 7.515555381774902, 8.054435729980469]
[8.084139823913574, 7.615833282470703, 7.7229838371276855, 8.84375, 9.09704303741455, 8.927285194396973, 6.68388891

,PosixTime,50,60,70,80,90,100,110,120,130,...,280,300,90 windiness,90 windiness 3 month,90 windiness 12 month,90 energiness,90 energiness 3 month,90 energiness 12 month,timestamp,year
UTCTimeZone,,,,,,,,,,,,,,,,,,,,,
2001-01-31,9.796446e+08,7.944086,8.333199,8.694758,9.064516,9.332931,9.611424,9.896505,10.078629,10.253763,...,11.560215,11.584543,1.053506,NaN,NaN,1.059391,NaN,NaN,2001-01-31,2001
2001-02-28,9.821934e+08,6.255506,6.492113,6.710417,6.931399,7.102679,7.278572,7.458333,7.580506,7.700149,...,8.637797,8.669642,0.884301,NaN,NaN,0.871574,NaN,NaN,2001-02-28,2001
2001-03-31,9.847422e+08,6.500269,6.748253,6.973522,7.203495,7.345968,7.497178,7.654570,7.742339,7.823791,...,7.973656,7.919892,0.901905,0.949901,NaN,0.891115,0.944390,NaN,2001-03-31,2001
2001-04-30,9.873774e+08,7.326250,7.572083,7.798056,8.022639,8.187361,8.353889,8.524861,8.636528,8.745694,...,9.486944,9.515833,0.983612,0.923893,NaN,0.981809,0.915521,NaN,2001-04-30,2001
2001-05-31,9.900126e+08,7.154570,7.418414,7.653495,7.893414,8.054436,8.217339,8.385080,8.488844,8.589382,...,9.338037,9.366398,1.011335,0.965411,NaN,1.012582,0.961606,NaN,2001-05-31,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,1.723808e+09,6.332393,6.573387,6.790188,7.018548,7.172446,7.334274,7.502016,7.610484,7.715322,...,8.380779,8.376344,1.015126,1.000843,0.983069,1.016790,1.000935,0.981207,2024-08-31,2024
2024-09-30,1.726443e+09,6.890833,7.209722,7.507500,7.810417,8.010834,8.217361,8.430695,8.554028,8.668056,...,9.323195,9.333889,1.073637,1.003019,0.993645,1.081738,1.003352,0.992946,2024-09-30,2024
2024-10-31,1.729078e+09,6.466667,6.746102,6.996774,7.253360,7.410887,7.575538,7.745430,7.837635,7.924597,...,8.256586,8.234139,0.896099,0.991097,0.988224,0.884670,0.990118,0.986929,2024-10-31,2024


[8.06478500366211, 7.513611316680908, 8.506586074829102, 8.08586311340332, 8.785080909729004, 9.501343727111816, 8.508749961853027, 7.374462127685547, 8.0024995803833, 7.363709926605225, 7.6043009757995605, 7.726110935211182]
[7.628055572509766, 8.06478500366211, 7.513611316680908, 8.506586074829102, 8.08586311340332, 8.785080909729004, 9.501343727111816, 8.508749961853027, 7.374462127685547, 8.0024995803833, 7.363709926605225, 7.6043009757995605]
[6.625, 7.628055572509766, 8.06478500366211, 7.513611316680908, 8.506586074829102, 8.08586311340332, 8.785080909729004, 9.501343727111816, 8.508749961853027, 7.374462127685547, 8.0024995803833, 7.363709926605225]
[6.672849655151367, 6.625, 7.628055572509766, 8.06478500366211, 7.513611316680908, 8.506586074829102, 8.08586311340332, 8.785080909729004, 9.501343727111816, 8.508749961853027, 7.374462127685547, 8.0024995803833]
[6.982083320617676, 6.672849655151367, 6.625, 7.628055572509766, 8.06478500366211, 7.513611316680908, 8.506586074829102, 8

[7.709408760070801, 7.098958492279053, 8.783064842224121, 9.102418899536133, 8.401805877685547, 8.381855010986328, 7.436110973358154, 6.889112949371338, 6.79448938369751, 7.941666603088379, 8.34852123260498, 7.860138893127441]
[7.632638931274414, 7.709408760070801, 7.098958492279053, 8.783064842224121, 9.102418899536133, 8.401805877685547, 8.381855010986328, 7.436110973358154, 6.889112949371338, 6.79448938369751, 7.941666603088379, 8.34852123260498]
[7.250671863555908, 7.632638931274414, 7.709408760070801, 7.098958492279053, 8.783064842224121, 9.102418899536133, 8.401805877685547, 8.381855010986328, 7.436110973358154, 6.889112949371338, 6.79448938369751, 7.941666603088379]
[7.284583568572998, 7.250671863555908, 7.632638931274414, 7.709408760070801, 7.098958492279053, 8.783064842224121, 9.102418899536133, 8.401805877685547, 8.381855010986328, 7.436110973358154, 6.889112949371338, 6.79448938369751]
[7.024059295654297, 7.284583568572998, 7.250671863555908, 7.632638931274414, 7.70940876007

,PosixTime,50,60,70,80,90,100,110,120,130,...,280,300,90 windiness,90 windiness 3 month,90 windiness 12 month,90 energiness,90 energiness 3 month,90 energiness 12 month,timestamp,year
UTCTimeZone,,,,,,,,,,,,,,,,,,,,,
2001-01-31,9.796446e+08,7.944086,8.333199,8.694758,9.064516,9.332931,9.611424,9.896505,10.078629,10.253763,...,11.560215,11.584543,1.053506,NaN,NaN,1.059391,NaN,NaN,2001-01-31,2001
2001-02-28,9.821934e+08,6.255506,6.492113,6.710417,6.931399,7.102679,7.278572,7.458333,7.580506,7.700149,...,8.637797,8.669642,0.884301,NaN,NaN,0.871574,NaN,NaN,2001-02-28,2001
2001-03-31,9.847422e+08,6.500269,6.748253,6.973522,7.203495,7.345968,7.497178,7.654570,7.742339,7.823791,...,7.973656,7.919892,0.901905,0.949901,NaN,0.891115,0.944390,NaN,2001-03-31,2001
2001-04-30,9.873774e+08,7.326250,7.572083,7.798056,8.022639,8.187361,8.353889,8.524861,8.636528,8.745694,...,9.486944,9.515833,0.983612,0.923893,NaN,0.981809,0.915521,NaN,2001-04-30,2001
2001-05-31,9.900126e+08,7.154570,7.418414,7.653495,7.893414,8.054436,8.217339,8.385080,8.488844,8.589382,...,9.338037,9.366398,1.011335,0.965411,NaN,1.012582,0.961606,NaN,2001-05-31,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,1.723808e+09,6.332393,6.573387,6.790188,7.018548,7.172446,7.334274,7.502016,7.610484,7.715322,...,8.380779,8.376344,1.015126,1.000843,0.983069,1.016790,1.000935,0.981207,2024-08-31,2024
2024-09-30,1.726443e+09,6.890833,7.209722,7.507500,7.810417,8.010834,8.217361,8.430695,8.554028,8.668056,...,9.323195,9.333889,1.073637,1.003019,0.993645,1.081738,1.003352,0.992946,2024-09-30,2024
2024-10-31,1.729078e+09,6.466667,6.746102,6.996774,7.253360,7.410887,7.575538,7.745430,7.837635,7.924597,...,8.256586,8.234139,0.896099,0.991097,0.988224,0.884670,0.990118,0.986929,2024-10-31,2024
